# Leverage AI Skill for comprehensive insights into your data


## Introduction

The AI Skill is a new experience in Microsoft Fabric that enables you to customize a generative AI expert on your data. Simply by selecting the relevant data sources in Fabric, you can create a Q&A chatbot that delivers insightful answers. This notebook will guide you on how to configure the AI Skill by providing additional context and details, in order to receive more comprehensive and contextually rich responses.

To illustrate, if a user queries about the best-selling product for a specific year within a retail dataset, AI Skill can be configured to provide more than just the top product. For example, it can display the top three best-selling products for that year and compare them to top sellers from previous years. This additional context helps users identify trends, such as shifts in top-selling products over time and their performance relative to other products.

This enriched Q&A experience can be achieved by integrating AI Skill with additional large language model (LLM) calls. The main steps include:

- Generate Additional Questions with LLM: Start by passing the user's question to the LLM. Request the LLM to generate multiple (e.g., three) supplementary questions that can provide more context or gather additional information related to the original question. For instance, if the user's question is `What is the best-selling product in 2019?`, the LLM might generate questions like `What are the top three best-selling products in 2019?`, `What were the best-selling products in 2018?`, or `What are the all-time best-selling products?`.

- Query AI Skill with All Questions: Submit both the original user's question and the additional questions generated by the LLM to the AI Skill. AI Skill will then query the dataset for each question separately, retrieving the relevant data for each question.

- Formulate a Comprehensive Answer with LLM: Use the results obtained from the AI Skill and pass this information to another LLM call. Instruct the LLM to craft a detailed response to the original question, incorporating the additional data provided.

By integrating AI Skill and LLMs, you can develop an enhanced solution that leverages  AI Skill's ability alongside the generality of LLMs to generate and execute SQL queries from natural language.


<img src="https://synapseaisolutionsa.blob.core.windows.net/public/AISkill/University_QnA.png" width="70%" height="70%" style="display: block; margin: 0 auto;">


The main steps in this notebook are:

1. Set up the lakehouse and add the relevant tables to the lakehouse
2. Create and publish AI Skill
3. Generate additional queries
4. Call the AI Skill function
5. Generate the conceptually rich final answer



## Prerequisites

1. To leverage Azure OpenAI in Fabric, you would require a [Fabric capacity (F64 or higher)](https://aka.ms/fabric/copilot-capacity).
2. Read about AI Skill specific requirements [here](https://aka.ms/fabric/ai-skill-prerequisites).

## Step 1: Set up the lakehouse and add the relevant tables

Create a lakehouse and then open a Fabric notebook. Attach the lakehouse to the notebook, and run the following code to add all tables to your lakehouse. Note that you will be using the AdventureWorks dataset which includes retail data with tables for sales, products, customers, promotions, etc.

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
base = "https://synapseaisolutionsa.blob.core.windows.net/public/AdventureWorks"

# load list of tables
df_tables = pd.read_csv(f"{base}/adventureworks.csv", names=["table"])

for table in (pbar := tqdm(df_tables['table'].values)):
    pbar.set_description(f"Uploading {table} to lakehouse")

    # download
    df = pd.read_parquet(f"{base}/{table}.parquet")

    # save as lakehouse table
    spark.createDataFrame(df).write.mode('overwrite').saveAsTable(table)

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 3, Finished, Available, Finished)

  0%|          | 0/28 [00:00<?, ?it/s]

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 21, Finished, Available, Finished)

## Step 2: Create and publish an AI Skill

Follow the steps below to create an AI Skill based on the lakehouse you created in Step 1.

1. Go to the Data Science experience and select AI Skill.

2. Enter a name to create an AI Skill.

<br>

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/AISkill/create-ai-skill.png" width="70%" height="70%" style="display: block; margin: 0 auto;">

<br>


3. Select the lakehouse you created above and select Connect.

<br>

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/AISkill/change-datasource.png" width="70%" height="70%" style="display: block; margin: 0 auto;">

<br>

4. You must then select the tables for which you want the AI Skill to have available access. From the left panel, select the tables **dimproduct**, **dimpromotion**, **factinternetsales** as these are the only tables needed for this notebook.

5. To improve the quality of the query generated by the AI Skill, provide the following the instructions as parts of the `Notes to model`.

    - When answering about a product, make sure to include the Product Name in `dimproduct` in the answer. 
    - Best selling product should be determined by sales volume, not sales amount. 
    - If you answer questions about quantities, make sure to include the quantity. 
    - If the user asks about promotion, note that `No Discount` appearing in `dimpromotion` is not actually a promotion. Always filter out the `No Discount` if user asks about promotion. 
    - If the question is generic and involves no finite entities such as "top 5", use a reasonable number less than 10, so that answer is not too large.

6. Validate the performance of the AI Skill by asking the following questions and reviewing both the final answers and the generated SQL queries used to retrieve them.

7. Click on the publish button to publish the AI Skill. Once published, navigate to the Settings, then click on Publishing to copy the provided URL to the cell below. You will use the URL to query the AI skill by making calls to the AI skill API in a this notebook. Learn more about how to use AI Skill programmatically [here](https://aka.ms/fabric/use-aiskill-programmatically).

<br>

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/AISkill/ai-select-publish.png" width="70%" height="70%" style="display: block; margin: 0 auto;">


<img src="https://synapseaisolutionsa.blob.core.windows.net/public/AISkill/initial-ai-skill-settings.png" width="70%" height="70%" style="display: block; margin: 0 auto;">


<img src="https://synapseaisolutionsa.blob.core.windows.net/public/AISkill/fabric-notebook-ai-skill-published-url-value.png" width="70%" height="70%" style="display: block; margin: 0 auto;">

<br>

In [ ]:
# AI Skill URL
aiskill_url = ""

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 4, Finished, Available, Finished)

##  Step 3: Generate additional queries

In this section, you'll explore how to transform your initial question into several related questions. By rephrasing the original question in different ways, you can gather more comprehensive information and provide a deeper understanding of the topic at hand.

##### Set up the function to generate additional queries

Below is a sample prompt that demonstrates how to generate multiple relevant questions based on the your initial question.

In the following function, you will see how to call a specific LLM model from OpenAI. The parameters—deployment ID, messages, temperature, and seed—are part of the experimental setup.

In this case, the prompt and question are passed directly into the messages, the temperature is set to 0, the seed is fixed, and the deployment ID is set to one of the supported models. For information on switching models, please refer to [Fabric AI Services](https://aka.ms/fabric/ai-services-overview).

In [ ]:
prompt_extract_questions = """You are a helpful analyst. You are given a user query, using the query create 3 relevant questions that will give more information around the *question being asked*. 
For example: If the question is, what is the top selling product in 2019? The rephrased questions could be: 
what were the top selling 3 products in 2019? what were the top 3 products sold in 2018? or what were the top 3 best-selling products across all years? 
Note that these questions will be independently used to query a SQL table.  Don't enumerate the questions and put each question in a new line."""

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 5, Finished, Available, Finished)

In [ ]:
import openai

# Given a user question, generates 3 more questions that will be used to obtain more context about the original question
def openAI_service_multiple_questions(query):
    response = openai.ChatCompletion.create(
        deployment_id='gpt-4-32k',
        messages=[
            {"role": "system", "content": prompt_extract_questions},
            {"role": "user", "content": query},
        ],
        temperature=0,
        seed=40,
    )
    return response

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 6, Finished, Available, Finished)

##### Provide user's specific question

You can now provide your own prompt, which will be fed as input to the `openAI_service_multiple_questions` function defined above to generate relevant questions. This allows you to customize the process and obtain more targeted information from the AI Skill.

In [ ]:
query = "What is the top selling product in 2013?"

response = openAI_service_multiple_questions(query)
answer = response.choices[0].message.content
print(f"{answer}")

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 7, Finished, Available, Finished)

What were the top 3 selling products in 2013?
What was the top selling product in 2012?
What were the top selling products across all years?


Once you have seen the generated questions by the `openAI_service_multiple_questions` function, you can break it into separate lines which helps to extract each question separately for easier processing later.

In [ ]:
questions = answer.split("\n")
print(questions)

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 8, Finished, Available, Finished)

['What were the top 3 selling products in 2013?', 'What was the top selling product in 2012?', 'What were the top selling products across all years?']


## Step 4: Call the AI Skill function

Once you have generated relevant questions, you move to next step to define the function `aiSkill` which would take three inputs

- `context`: the notes for the model in the AI Skill
- `question`: user's question
- `aiskill_url`: the URL of the published AI Skill

in order to generate the SQL query based on these inputs. It then outputs the answer in a JSON format within the `response`.

In [ ]:
from synapse.ml.mlflow import get_mlflow_env_config
from tenacity import retry, stop_after_attempt
import requests
import json

@retry(stop=stop_after_attempt(3))  # Retry up to 3 times
def aiSkill(context, question, aiskill_url):

    configs = get_mlflow_env_config()   
    headers = {
        "Authorization": f"Bearer {configs.driver_aad_token}",
        "Content-Type": "application/json; charset=utf-8"
    }

    text = {
        'userQuestion': question, 
        'modelBehavior' : {
            'enableBlockAdditionalContextByLength': False,
        },
        'additionalContext': context # notes to the model
    }

    response = requests.post(aiskill_url, headers=headers, data = json.dumps(text))

    response = json.loads(response.content)

    # If the AISkill didn't generate a good SQL query, it will throw an error
    if "ResultRows" not in response.keys():
        raise ValueError(response["message"])

    return response

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 9, Finished, Available, Finished)

In the following, you'll provide relevant context or notes to the AI Skill. Note that this additional information - which is usually provided by the subject matter expert who has good knowledge of the data - helps the AI Skill determine the appropriate columns to use and generate accurate and relevant outputs. 

In [ ]:
notes = """ \
- When answering about a product, make sure to include the Product Name in dimproduct in the answer. 
- Best selling product should be determined by sales volume, not sales amount. 
- If you answer questions about quantities, make sure to include the quantity. 
- If the user asks about promotion, note that "No Discount" appearing in dimpromotion is not actually a promotion. Always filter out the "No Discount" if user asks about promotion. 
- If the question is generic and involves no finite entities such as "top 5", use a reasonable number less than 10, so that answer is not too large.
"""

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 10, Finished, Available, Finished)

Consolidate all outputs from the AI Skill by combining the results generated for each rephrased question into a single, unified set of answers. This ensures that you have a comprehensive view of all the responses and can analyze or utilize them collectively.

In [ ]:
# Run a loop over the generated questions and get an answer from AI Skill
def get_context_from_aiSkill(questions):
    ai_skill_response = []
    ai_skill_sql = []
    for question in questions:
        response = aiSkill(notes, question, aiskill_url)
        ai_skill_response.append(f"headers: {response['ResultHeaders']}, rows: {response['ResultRows']}")
        ai_skill_sql.append(response['executedSQL'])
    return ai_skill_response, ai_skill_sql

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 11, Finished, Available, Finished)

In [ ]:
questions.append(query)
ai_skill_response, ai_skill_sql = get_context_from_aiSkill(questions)

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 12, Finished, Available, Finished)

You can review the SQL query generated for the each of the rephrased question. This allows you to verify and understand the SQL code created by the AI Skill in response to each specific question.

In [ ]:
import pandas as pd

# Adjust Pandas display options to prevent truncation of long strings
pd.set_option('display.max_colwidth', None)  # Set to 'None' to display all contents without truncation

def display_ai_skill_data(questions, ai_skill_sql, ai_skill_response):
    """
    This function takes questions, AI Skill SQL queries, and AI Skill responses, and 
    displays them in a table with three columns: 'Question', 'SQL Query', and 'AI Skill Response'.
    
    Args:
    questions (list): List of questions.
    ai_skill_sql (list): List of SQL queries generated by AI Skill.
    ai_skill_response (list): List of responses generated by AI Skill.
    
    Returns:
    DataFrame: A pandas DataFrame containing the questions, SQL queries, and responses.
    """
    # Create DataFrame
    df = pd.DataFrame({
        'Question': questions,
        'SQL Query': ai_skill_sql,
        'AI Skill Response': ai_skill_response
    })
    
    # Display the DataFrame
    return df

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 13, Finished, Available, Finished)

In [ ]:
questions = [
    questions[0],
    questions[1],
    questions[2]
]

ai_skill_sql = [
    ai_skill_sql[0],
    ai_skill_sql[1],
    ai_skill_sql[2]
]

ai_skill_response = [
    ai_skill_response[0],
    ai_skill_response[1],
    ai_skill_response[2]
]

# Call the function and display the data
display_ai_skill_data(questions, ai_skill_sql, ai_skill_response)

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 14, Finished, Available, Finished)

,Question,SQL Query,AI Skill Response
0,What were the top 3 selling products in 2013?,"SELECT TOP 3 \n dp.EnglishProductName AS ProductName,\n SUM(fis.OrderQuantity) AS TotalQuantity\nFROM \n dbo.factinternetsales fis\nJOIN \n dbo.dimproduct dp ON fis.ProductKey = dp.ProductKey\nWHERE \n YEAR(fis.OrderDate) = 2013\nGROUP BY \n dp.EnglishProductName\nORDER BY \n TotalQuantity DESC;","headers: ['ProductName', 'TotalQuantity'], rows: [['Water Bottle - 30 oz.', 4080], ['Patch Kit/8 Patches', 3026], ['Mountain Tire Tube', 2926]]"
1,What was the top selling product in 2012?,SELECT TOP 1 dp.EnglishProductName AS TopSellingProduct\nFROM dbo.factinternetsales fis\nJOIN dbo.dimproduct dp ON fis.ProductKey = dp.ProductKey\nWHERE YEAR(fis.OrderDate) = 2012\nGROUP BY dp.EnglishProductName\nORDER BY SUM(fis.OrderQuantity) DESC;,"headers: ['TopSellingProduct'], rows: [['Mountain-200 Black, 46']]"
2,What were the top selling products across all years?,"SELECT TOP 5 dp.EnglishProductName AS TopSellingProduct, SUM(fis.OrderQuantity) AS TotalQuantity\nFROM dbo.factinternetsales fis\nJOIN dbo.dimproduct dp ON fis.ProductKey = dp.ProductKey\nGROUP BY dp.EnglishProductName\nORDER BY TotalQuantity DESC;","headers: ['TopSellingProduct', 'TotalQuantity'], rows: [['Water Bottle - 30 oz.', 4244], ['Patch Kit/8 Patches', 3191], ['Mountain Tire Tube', 3095], ['Road Tire Tube', 2376], ['Sport-100 Helmet, Red', 2230]]"


In the following, you will integrate the questions and answers from the AI Skill into the context.

In [ ]:
def get_context_to_answer_with_llm(query, rephrased_query, ai_skill_response):
    context = ""
    for index, question in enumerate(questions):
        context += f"rephrased question: {question} \n" + f"header and rows of the table: {ai_skill_response[index]}\n" 

    context += f"user query: {query}"
    return context

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 15, Finished, Available, Finished)

In [ ]:
context = get_context_to_answer_with_llm(query, questions, ai_skill_response)

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 16, Finished, Available, Finished)

## Step 5: Generate conceptually rich answer

Consolidating the initial question, all additional questions, and the AI Skill response into the final LLM-generated answer. This process results in a cohesive, comprehensive answer produced by LLM, integrating all relevant information into a final, unified response.

In [ ]:
prompt_final_answer = """You are an expert analyst. Given a user query, rephrased query asking more details, and the relevant context that is from an output of a SQL 
table presented as headers and corresponding rows, answer the user question with details. 
Formulate your response to directly answer the original user query, while adding details with the answers to the rephrased queries. 
Do not mention things like 'Based on the data provided' or anything about the data, just answer the user question."""

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 17, Finished, Available, Finished)

In [ ]:
def openAI_service_generate_answer(query, rephrased_query, ai_skill_response):
    context = get_context_to_answer_with_llm(query, rephrased_query, ai_skill_response)
    response = openai.ChatCompletion.create(
        deployment_id='gpt-4-32k', 
        messages=[
            {"role": "system", "content": prompt_final_answer },
            {"role": "user", "content": f"Use the information provided in the context headers and rows: {context} and write a descriptive report assessing all the table information, then concluding with the answer to the user query." },
        ],
        temperature=0)

    return response

response = openAI_service_generate_answer(query, questions, ai_skill_response)
answer = response.choices[0].message.content
print(f"{answer}")

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 18, Finished, Available, Finished)

The top selling products in 2013 were the Water Bottle - 30 oz., Patch Kit/8 Patches, and Mountain Tire Tube, with total quantities of 4080, 3026, and 2926 respectively. In 2012, the top selling product was the Mountain-200 Black, 46. 

When we look at the top selling products across all years, the Water Bottle - 30 oz. leads the pack with a total quantity of 4244. Following closely are the Patch Kit/8 Patches and Mountain Tire Tube with total quantities of 3191 and 3095 respectively. Other notable top sellers include the Road Tire Tube and Sport-100 Helmet, Red, with total quantities of 2376 and 2230 respectively.

In conclusion, the top selling product in 2013 was the Water Bottle - 30 oz. with a total quantity of 4080.


Now that you have a clear understanding of how the framework operates, you can apply it to additional examples. This will help you see how the framework handles various scenarios and further clarify its functionality. You can call the `openAI_service_multiple_questions` function again to generate multiple related questions through rephrasing the main query.

In [ ]:
query = "What are the unique product pairs that are most frequently purchased together in 2013?"
print(f"Query: {query}")

response = openAI_service_multiple_questions(query)
answer = response.choices[0].message.content
# print(f"{answer}")
questions = answer.split("\n")
questions.append(query)
# print(questions)

ai_skill_response = []
ai_skill_sql = []
for question in questions:
    response = aiSkill(notes, question, aiskill_url)
    ai_skill_response.append(f"headers: {response['ResultHeaders']}, rows: {response['ResultRows']}")
    ai_skill_sql.append(response['executedSQL'])

response = openAI_service_generate_answer(query, questions, ai_skill_response)
answer = response.choices[0].message.content
print(f"Response: {answer}")

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 19, Finished, Available, Finished)

Query: What are the unique product pairs that are most frequently purchased together in 2013?
Response: In 2013, the most frequently purchased product pair was the 'Water Bottle - 30 oz.' and the 'Mountain Bottle Cage', with a purchase frequency of 1626. This was closely followed by the 'Water Bottle - 30 oz.' and the 'Road Bottle Cage', which was purchased together 1507 times. The third most common product pair was the 'Mountain Tire Tube' and the 'HL Mountain Tire', with a purchase count of 919. The 'Touring Tire Tube' and 'Touring Tire' were also frequently bought together, with a purchase frequency of 760. Lastly, the 'Patch Kit/8 Patches' and 'Mountain Tire Tube' were purchased together 739 times.


In [ ]:
query = "Which promotion had the highest sales of products in terms of sales volume in 2013?"
print(f"Query: {query}")

response = openAI_service_multiple_questions(query)
answer = response.choices[0].message.content
# print(f"{answer}")
questions = answer.split("\n")
questions.append(query)
# print(questions)

ai_skill_response = []
ai_skill_sql = []
for question in questions:
    response = aiSkill(notes, question, aiskill_url)
    ai_skill_response.append(f"headers: {response['ResultHeaders']}, rows: {response['ResultRows']}")
    ai_skill_sql.append(response['executedSQL'])

response = openAI_service_generate_answer(query, questions, ai_skill_response)
answer = response.choices[0].message.content
print(f"Response: {answer}")

StatementMeta(, 41947b06-61e4-4d32-b0a9-dc833847a498, 20, Finished, Available, Finished)

Query: Which promotion had the highest sales of products in terms of sales volume in 2013?
Response: The top three promotions that had the highest sales of products in terms of sales volume in 2013 were the 'Volume Discount 11 to 14' with 1747 units sold, followed by the 'Touring-3000 Promotion' with 20 units sold, and the 'Touring-1000 Promotion' with 13 units sold. 

In 2012, the promotion that had the highest sales of products in terms of sales volume was the 'Volume Discount 11 to 14' with 367 units sold. 

When considering all years, the promotion that had the highest sales of products in terms of sales volume was the 'Volume Discount 11 to 14'. 

However, in 2013, the promotion that had the highest sales of products in terms of sales volume was the 'No Discount' promotion. This indicates that despite the success of the 'Volume Discount 11 to 14' promotion in previous years and overall, customers in 2013 were more inclined to purchase products without any promotional discounts.


##### Conclusion

By following the steps outlined in this notebook, you can now create a powerful, customized AI Skill that provides comprehensive and contextually rich responses to your queries. Integrating AI Skill with LLMs allows for deeper insights by generating additional relevant questions and offering more detailed answers. This process enhances decision-making and provides users with a more enriched, data-driven experience using Microsoft Fabric.